In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

data = pd.read_csv('/kaggle/input/bankOffers/Transaction_Naration_Data_Set.csv')  # Replace 'transactions.csv' with the actual filename

# Perform any necessary preprocessing steps, such as removing noise or irrelevant characters


/kaggle/input/bankOffers/Transaction_Naration_Data_Set.csv


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# data.head(20)
# data['NARATION'] = data['NARATION'].str.lower()
# data.head()

# print(data['AMOUNT'].max())
# print(data['AMOUNT'].min())

# distinct_count = data['AC_ID'].nunique()  # Replace 'column_name' with the actual column name

# # Print the number of distinct values
# print("Number of distinct values:", distinct_count)
# print(len(data))

# Group the rows based on 'AC_ID'
# grouped_data = data.groupby('AC_ID')
# type(grouped_data)

# # Create a new DataFrame with the grouped data
# grouped_df = pd.DataFrame({'AC_ID': grouped_data.groups.keys(), 'GroupedRows': grouped_data.groups.values()})

# # Print the resulting DataFrame
# print(grouped_df.head())

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

data['NARATION'] = data['NARATION'].str.replace('[^A-Za-z0-9]+', ' ', regex=True)
data.head()

# Perform tokenization, stemming, and stop word removal
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
data['NARATION'].fillna('', inplace=True)

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    return tokens

# Apply the preprocessing function to the 'NARATION' column
data['TOKENS'] = data['NARATION'].apply(preprocess_text)

# Print the preprocessed tokens
print(data.head())

   Unnamed: 0    AC_ID    AMOUNT   DOC_DATE  \
0           0  3644877  342800.0  01-NOV-22   
1           1  1175256  342800.0  01-NOV-22   
2           2  2298226    1500.0  01-NOV-22   
3           3  4066427    1500.0  01-NOV-22   
4           4  1978564    2550.0  01-NOV-22   

                                            NARATION   DR_CR  \
0                                      Cash Withdraw   Debit   
1                       Cash Withdrawal From A C No   Credit   
2    Cash withdraw from Micro Merchant point MM A C    Debit   
3  Credit Against Merchant Cash Withdraw Service ...  Credit   
4    Cash withdraw from Micro Merchant point MM A C    Debit   

                                              TOKENS  
0                                   [cash, withdraw]  
1                   [cash, withdraw, from, A, C, No]  
2  [cash, withdraw, micro, merchant, point, MM, A...  
3  [credit, against, merchant, cash, withdraw, se...  
4  [cash, withdraw, micro, merchant, point, MM, A...  


In [4]:
# cls = MiniBatchKMeans(n_clusters=5, random_state=random_state)
# cls.fit()
data.head()
vec = TfidfVectorizer(stop_words="english")
vec.fit(data.NARATION.values)
keywords = vec.transform(data.NARATION.values)